In [1]:
pip install pandas

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install spotipy

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install urllib3 --upgrade 
%pip install requests --upgrade 
%pip install spotipy --upgrade

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [4]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import os
import numpy as np
import timeit
import pandas as pd
from operator import itemgetter
import time
import datetime as dt
import json

os.environ['SPOTIPY_CLIENT_ID'] = '3191e7e8b04e46c1af64f49bcdd020be'
os.environ['SPOTIPY_CLIENT_SECRET'] = '88dc925eff0d4006bb6d0b9b2401ac59'

In [5]:
auth_manager = SpotifyClientCredentials()
sp = spotipy.Spotify(auth_manager=auth_manager)

In [6]:
# calculate artist concentration value for a playlist
# low value means high concentration - maybe rename to artist diversity?
# input: 2D array of artists for each song
# output: real value between 0 and 1
def get_artist_conc(artists):
    print('start get_artist_conc')
    unique_artists = len(set(artists))
    total_appearances = len(artists)
    artist_conc = unique_artists / total_appearances
    print('end get_artist_conc')
    return artist_conc

In [7]:
MS_IN_DAY = 86400
def get_artists_and_dates(uris):
    print('start get_artists_and_dates')
    # do the batching above this- here we assume that there are no more than 50 URIs
    # i could put an assert statement here but it would slow the code down
    # assert(len(uris) <= 50)
    artists = []
    dates = []
    none_uris = []
    print('getting tracks')
    tracks = sp.tracks(uris)['tracks']
    print('got tracks, processing')
    for uri, track in zip(uris, tracks):
        if track is None:
            #print(track)
            none_uris.append(uri)
            print(sp.tracks(uri))# delete this before running
        else:
            for artist in track['artists']:
                artists.append(artist['name'])
            date = track['album']['release_date']
            while len(date) < 10:
                date += '-01'
            ms_date = dt.datetime.timestamp(dt.datetime.strptime(date,"%Y-%m-%d"))
            dates.append(ms_date / MS_IN_DAY)
            
    print('end get_artists_and_dates')
    return artists, dates, none_uris

In [8]:
# get all audio features for a given playlist
def get_playlist_audio_features_batch(tracks):
    print('start get_playlist_audio_features_batch')
    tracks = pd.DataFrame(tracks)
    playlist_uris = tracks['track_uri']
    none_uris = []

    remainder = len(tracks) % 50
    artists, dates = [], []
    if remainder > 0:
        artists, dates, none_uris_iter = get_artists_and_dates(playlist_uris[:remainder])
        none_uris += none_uris_iter
    while remainder < len(tracks):
        next_artists, next_dates, none_uris_iter = get_artists_and_dates(playlist_uris[remainder:remainder+50])
        artists += next_artists
        dates += next_dates
        remainder += 50
        none_uris += none_uris_iter

    # remove none uris from playlist_uris
    none_uris = set(none_uris)
    playlist_uris = list(set(playlist_uris).difference(none_uris))
    #none_uris = list(none_uris)


    remainder = len(tracks) % 100
    all_features = []
    if remainder > 0: 
        all_features = sp.audio_features(playlist_uris[:remainder])
    while remainder < len(tracks):
        all_features += sp.audio_features(playlist_uris[remainder:remainder+100])
        remainder += 100
    
    print('end get_playlist_audio_features_batch')
    return all_features, artists, dates, none_uris

In [9]:
# calculate aggregate audio features for a playlist
# currently only focused on audio features, I'll deal with artist and release date later
def get_playlist_aggregate_features_v2(tracks):
    print('start get_playlist_aggregate_features_v2')
    
    all_features, artists, dates = get_playlist_audio_features_batch(tracks)
    all_features = pd.DataFrame(all_features)

    feature_list = ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']
    aggregate_features = {}
    for feature in feature_list:
        all_by_feature = all_features[feature]
        aggregate_features[feature] = np.mean(all_by_feature)
        aggregate_features[feature+'_var'] = np.var(all_by_feature)
    aggregate_features['date'] = np.mean(dates)
    aggregate_features['date_var'] = np.var(dates)
    aggregate_features['artist_conc'] = get_artist_conc(artists)
    print('end get_playlist_aggregate_features_v2')
    return aggregate_features

In [10]:
# build the aggregate dataset!
def build_dataset_slice(playlists):
    aggregate_dataset = []
    for playlist in playlists:
        aggregate_dataset.append(get_playlist_aggregate_features_v2(playlist['tracks']))
    return aggregate_dataset

In [11]:
slice_prefix = 'mpd_slice_'
slice_zeros = '000_'
slice_suffix = '999.json'

db_prefix = 'zdb_slice_'
db_zeros = '000_'
db_suffix = '999.csv'

In [12]:
def build_full_zdb(start, end):
    for i in range(NUM_SLICES):
        i = str(i)
        print('Starting slice', i)
        #with open('spotify_million_playlist_dataset/data/'+slice_prefix + i + slice_zeros + i + slice_suffix) as json_file:
        with open(slice_prefix + i + slice_zeros + i + slice_suffix) as json_file:
            data = json.load(json_file)
        print('Opened file for slice', i)
        dataset_slice = pd.DataFrame(build_dataset_slice(data['playlists'][0:10]))
        print('Built dataset for slice', i)
        dataset_slice.to_csv(r''+db_prefix + i + db_zeros + i + db_suffix)
        print('Completed slice', i)

In [13]:
START_INDEX = 0
NUM_SLICES = 1
build_full_zdb(START_INDEX, NUM_SLICES)

Starting slice 0
Opened file for slice 0
start get_playlist_aggregate_features_v2
start get_playlist_audio_features_batch
start get_artists_and_dates
getting tracks
got tracks, processing
end get_artists_and_dates
start get_artists_and_dates
getting tracks
got tracks, processing
end get_artists_and_dates
end get_playlist_audio_features_batch


ValueError: too many values to unpack (expected 3)